# Identifying Bias in SpaCy word vectors

In [1]:
import spacy
from scipy import spatial
import tqdm

In [2]:
nlp = spacy.load('en_core_web_lg')

In [8]:
# Add in roles and gender pairs to compare
neutral_roles = ["doctor", "teacher", "scientist", "dancer", "builder", "boss", "engineer", "professor"]
gender_pairs = [
    ("male", "female"),
    ("man", "woman")
]

In [4]:
def cosine_similarity(term1: str, term2: str) -> float:
    """
    Gets the cosine similarity between two vectors
    """
    return 1 - spatial.distance.cosine(term1, term2)

In [5]:
def get_diff_roles(roles1: list, roles2: list) -> list:
    """
    Gets difference in cosine similarity of two role lists, returning a list of role-similarity difference pairs
    """
    results = {}
    roles1_dict = dict(roles1)
    roles2_dict = dict(roles2)
    keys = set(roles1_dict.keys()) | set(roles2_dict.keys())
    for k in tqdm.tqdm(keys, desc="Getting similarity difference between roles", unit="roles"):
        s1 = roles1_dict.get(k, 0)
        s2 = roles2_dict.get(k, 0)
        results[k] = ((s1, s1-s2), (s2, s2-s1))  # 1st similarity, 2nd similarity, similarity diff
    
    return results.items()


def get_top_roles(roles: list, similarity_index: int, top_n: int = 20, min_sim: float = 0.5) -> str:
    """
    Gets top N roles from similarity comparison list in get_diff_roles
    """
    return list(sorted(filter(lambda x: x[1][similarity_index][0 ] >= min_sim, roles), key=lambda x: x[1][similarity_index][0], reverse=True)[:top_n])

In [6]:
from IPython.display import Markdown, display


def get_table_row(role: list, similarity_index: int):
    """
    Formats the nested inner table for display_roles_table
    """
    return f'<tr><td>{role[0]}</td><td>{100*role[1][similarity_index][0]:.1f}% ({100*role[1][similarity_index][1]:.1f}%)</td></tr>'


def display_roles_table(role: str, m_label: str, f_label: str, m_roles: list, f_roles: list, diff_roles: list) -> str:
    m_rows = "<table>" + "".join([get_table_row(role, similarity_index=0) for role in m_roles]) + "</table>"
    f_rows = "<table>" + "".join([get_table_row(role, similarity_index=1) for role in f_roles]) + "</table>"
    form_row = f'<tr><td>{role}</td><td>{m_rows}</td><td>{f_rows}</td></tr>'
    form_table = f'<table><tr><th>Role</th><th>{m_label}</th><th>{f_label}</th></tr>' + form_row + "</table>"
    display(Markdown(form_table))

In [7]:
for role in neutral_roles:
    # Create role vector
    role_vect = nlp.vocab[role].vector
    for m, f in gender_pairs:
        # Create gender vectors
        m_vect = nlp.vocab[m].vector
        f_vect = nlp.vocab[f].vector
        m_role = role_vect - f_vect + m_vect
        f_role = role_vect - m_vect + f_vect
        
        # Find cosine similarity of all words to these words
        m_role_sims = []
        f_role_sims = []
        for word in tqdm.tqdm(nlp.vocab, desc=f"Looking for gender bias in {role!r} for {m!r} and {f!r}", unit=" words"):
            if word.has_vector and word.is_lower and word.is_alpha:
                m_role_sims.append((word.text, cosine_similarity(m_role, word.vector)))
                f_role_sims.append((word.text, cosine_similarity(f_role, word.vector)))
        
        # Find difference between genders in roles
        diff_roles = get_diff_roles(m_role_sims, f_role_sims)
        top_m_roles = get_top_roles(diff_roles, similarity_index=0, min_sim=0.0)
        top_f_roles = get_top_roles(diff_roles, similarity_index=1, min_sim=0.0)
        
        # Display data as table
        display_roles_table(role, m, f, top_m_roles, top_f_roles, diff_roles)

Looking for gender bias in 'doctor' for 'male' and 'female': 100%|█████████▉| 1343765/1344233 [00:24<00:00, 55832.35 words/s]
Getting similarity difference between roles: 100%|██████████| 247512/247512 [00:00<00:00, 499711.22roles/s]


<table><tr><th>Role</th><th>male</th><th>female</th></tr><tr><td>doctor</td><td><table><tr><td>doctor</td><td>93.2% (0.1%)</td></tr><tr><td>physician</td><td>75.1% (2.4%)</td></tr><tr><td>doctors</td><td>72.6% (0.4%)</td></tr><tr><td>surgeon</td><td>67.4% (3.5%)</td></tr><tr><td>pharmacist</td><td>66.3% (-1.4%)</td></tr><tr><td>medical</td><td>64.7% (-1.2%)</td></tr><tr><td>medicine</td><td>64.2% (0.6%)</td></tr><tr><td>medication</td><td>63.9% (2.6%)</td></tr><tr><td>patient</td><td>62.8% (1.0%)</td></tr><tr><td>pediatrician</td><td>62.1% (0.5%)</td></tr><tr><td>medications</td><td>61.3% (5.5%)</td></tr><tr><td>psychiatrist</td><td>60.9% (-1.1%)</td></tr><tr><td>meds</td><td>60.6% (4.6%)</td></tr><tr><td>cardiologist</td><td>60.0% (4.6%)</td></tr><tr><td>dentist</td><td>59.5% (1.7%)</td></tr><tr><td>clinic</td><td>59.4% (-0.3%)</td></tr><tr><td>surgery</td><td>59.2% (5.0%)</td></tr><tr><td>urologist</td><td>59.1% (14.0%)</td></tr><tr><td>dermatologist</td><td>58.6% (3.0%)</td></tr><tr><td>prescribe</td><td>57.7% (3.9%)</td></tr></table></td><td><table><tr><td>doctor</td><td>93.1% (-0.1%)</td></tr><tr><td>physician</td><td>72.6% (-2.4%)</td></tr><tr><td>doctors</td><td>72.3% (-0.4%)</td></tr><tr><td>nurse</td><td>70.9% (13.6%)</td></tr><tr><td>pharmacist</td><td>67.7% (1.4%)</td></tr><tr><td>medical</td><td>65.9% (1.2%)</td></tr><tr><td>surgeon</td><td>63.9% (-3.5%)</td></tr><tr><td>medicine</td><td>63.6% (-0.6%)</td></tr><tr><td>psychiatrist</td><td>62.0% (1.1%)</td></tr><tr><td>patient</td><td>61.9% (-1.0%)</td></tr><tr><td>midwife</td><td>61.9% (15.4%)</td></tr><tr><td>pediatrician</td><td>61.7% (-0.5%)</td></tr><tr><td>medication</td><td>61.4% (-2.6%)</td></tr><tr><td>hospital</td><td>60.0% (3.8%)</td></tr><tr><td>clinic</td><td>59.7% (0.3%)</td></tr><tr><td>gynecologist</td><td>59.5% (2.3%)</td></tr><tr><td>oncologist</td><td>58.8% (2.8%)</td></tr><tr><td>dentist</td><td>57.8% (-1.7%)</td></tr><tr><td>veterinarian</td><td>57.7% (4.4%)</td></tr><tr><td>neurologist</td><td>56.4% (-0.8%)</td></tr></table></td></tr></table>

Looking for gender bias in 'doctor' for 'man' and 'woman': 100%|█████████▉| 1343765/1344233 [00:23<00:00, 57964.39 words/s]
Getting similarity difference between roles: 100%|██████████| 247512/247512 [00:00<00:00, 476519.92roles/s]


<table><tr><th>Role</th><th>man</th><th>woman</th></tr><tr><td>doctor</td><td><table><tr><td>doctor</td><td>78.5% (-6.4%)</td></tr><tr><td>physician</td><td>62.8% (-4.0%)</td></tr><tr><td>doctors</td><td>59.2% (-8.4%)</td></tr><tr><td>medicine</td><td>56.9% (1.3%)</td></tr><tr><td>surgeon</td><td>56.3% (-2.6%)</td></tr><tr><td>pharmacist</td><td>54.8% (-7.7%)</td></tr><tr><td>medical</td><td>54.4% (-5.5%)</td></tr><tr><td>he</td><td>53.9% (29.0%)</td></tr><tr><td>man</td><td>53.7% (35.7%)</td></tr><tr><td>psychiatrist</td><td>52.8% (-2.3%)</td></tr><tr><td>medication</td><td>52.5% (-4.7%)</td></tr><tr><td>meds</td><td>52.2% (1.7%)</td></tr><tr><td>vet</td><td>51.1% (12.7%)</td></tr><tr><td>him</td><td>51.1% (24.8%)</td></tr><tr><td>cardiologist</td><td>50.9% (0.2%)</td></tr><tr><td>dentist</td><td>50.6% (-1.9%)</td></tr><tr><td>drugs</td><td>50.2% (8.0%)</td></tr><tr><td>neurologist</td><td>50.1% (0.3%)</td></tr><tr><td>medications</td><td>49.6% (-3.5%)</td></tr><tr><td>patient</td><td>49.0% (-10.8%)</td></tr></table></td><td><table><tr><td>doctor</td><td>84.8% (6.4%)</td></tr><tr><td>nurse</td><td>70.2% (30.1%)</td></tr><tr><td>doctors</td><td>67.6% (8.4%)</td></tr><tr><td>physician</td><td>66.8% (4.0%)</td></tr><tr><td>pregnant</td><td>66.6% (45.0%)</td></tr><tr><td>gynecologist</td><td>62.9% (25.0%)</td></tr><tr><td>woman</td><td>62.7% (46.8%)</td></tr><tr><td>pharmacist</td><td>62.5% (7.7%)</td></tr><tr><td>midwife</td><td>61.0% (29.2%)</td></tr><tr><td>medical</td><td>59.9% (5.5%)</td></tr><tr><td>pediatrician</td><td>59.9% (11.8%)</td></tr><tr><td>patient</td><td>59.8% (10.8%)</td></tr><tr><td>pregnancy</td><td>59.7% (37.9%)</td></tr><tr><td>surgeon</td><td>58.9% (2.6%)</td></tr><tr><td>clinic</td><td>57.5% (11.2%)</td></tr><tr><td>obstetrician</td><td>57.5% (20.8%)</td></tr><tr><td>medication</td><td>57.3% (4.7%)</td></tr><tr><td>medicine</td><td>55.6% (-1.3%)</td></tr><tr><td>hospital</td><td>55.4% (9.3%)</td></tr><tr><td>nurses</td><td>55.4% (24.8%)</td></tr></table></td></tr></table>

Looking for gender bias in 'teacher' for 'male' and 'female': 100%|█████████▉| 1343765/1344233 [00:24<00:00, 55889.75 words/s]
Getting similarity difference between roles: 100%|██████████| 247512/247512 [00:00<00:00, 491449.12roles/s]


...

<table><tr><th>Role</th><th>man</th><th>woman</th></tr><tr><td>engineer</td><td><table><tr><td>engineer</td><td>82.6% (3.0%)</td></tr><tr><td>engineering</td><td>63.3% (6.5%)</td></tr><tr><td>engineers</td><td>62.6% (6.3%)</td></tr><tr><td>architect</td><td>53.5% (3.6%)</td></tr><tr><td>technician</td><td>53.4% (-5.9%)</td></tr><tr><td>electrician</td><td>50.5% (5.4%)</td></tr><tr><td>geologist</td><td>49.9% (6.2%)</td></tr><tr><td>contractor</td><td>49.5% (2.3%)</td></tr><tr><td>surveyor</td><td>49.1% (8.8%)</td></tr><tr><td>mechanic</td><td>49.1% (3.3%)</td></tr><tr><td>inventor</td><td>48.1% (7.9%)</td></tr><tr><td>scientist</td><td>47.8% (-0.2%)</td></tr><tr><td>foreman</td><td>46.9% (13.3%)</td></tr><tr><td>technical</td><td>46.0% (6.3%)</td></tr><tr><td>machinist</td><td>45.9% (4.8%)</td></tr><tr><td>programmer</td><td>45.8% (2.3%)</td></tr><tr><td>aerospace</td><td>45.5% (2.1%)</td></tr><tr><td>journeyman</td><td>44.7% (14.3%)</td></tr><tr><td>draftsman</td><td>44.3% (4.5%)</td></tr><tr><td>manager</td><td>44.0% (2.1%)</td></tr></table></td><td><table><tr><td>engineer</td><td>79.7% (-3.0%)</td></tr><tr><td>technician</td><td>59.2% (5.9%)</td></tr><tr><td>engineering</td><td>56.8% (-6.5%)</td></tr><tr><td>engineers</td><td>56.3% (-6.3%)</td></tr><tr><td>technologist</td><td>52.7% (14.0%)</td></tr><tr><td>consultant</td><td>51.6% (12.0%)</td></tr><tr><td>supervisor</td><td>50.4% (9.6%)</td></tr><tr><td>worker</td><td>49.9% (20.6%)</td></tr><tr><td>architect</td><td>49.8% (-3.6%)</td></tr><tr><td>assistant</td><td>49.2% (11.8%)</td></tr><tr><td>scientist</td><td>48.1% (0.2%)</td></tr><tr><td>accountant</td><td>47.9% (10.2%)</td></tr><tr><td>woman</td><td>47.8% (54.9%)</td></tr><tr><td>contractor</td><td>47.2% (-2.3%)</td></tr><tr><td>nurse</td><td>47.0% (41.0%)</td></tr><tr><td>designer</td><td>47.0% (16.6%)</td></tr><tr><td>educator</td><td>46.6% (21.1%)</td></tr><tr><td>researcher</td><td>46.1% (13.3%)</td></tr><tr><td>freelance</td><td>46.0% (18.6%)</td></tr><tr><td>mechanic</td><td>45.8% (-3.3%)</td></tr></table></td></tr></table>

Looking for gender bias in 'professor' for 'male' and 'female': 100%|█████████▉| 1343765/1344233 [00:26<00:00, 50794.04 words/s]
Getting similarity difference between roles: 100%|██████████| 247512/247512 [00:00<00:00, 456328.33roles/s]


<table><tr><th>Role</th><th>male</th><th>female</th></tr><tr><td>professor</td><td><table><tr><td>professor</td><td>93.7% (-0.7%)</td></tr><tr><td>emeritus</td><td>69.3% (2.5%)</td></tr><tr><td>lecturer</td><td>67.9% (-7.6%)</td></tr><tr><td>dean</td><td>63.9% (-2.7%)</td></tr><tr><td>associate</td><td>63.8% (-0.7%)</td></tr><tr><td>researcher</td><td>63.7% (-2.4%)</td></tr><tr><td>professors</td><td>62.7% (-1.1%)</td></tr><tr><td>scholar</td><td>62.4% (-2.1%)</td></tr><tr><td>sociology</td><td>62.0% (0.2%)</td></tr><tr><td>scientist</td><td>61.3% (-2.5%)</td></tr><tr><td>graduate</td><td>61.2% (-4.5%)</td></tr><tr><td>doctorate</td><td>60.8% (-0.0%)</td></tr><tr><td>doctoral</td><td>60.1% (-2.5%)</td></tr><tr><td>faculty</td><td>59.0% (-4.4%)</td></tr><tr><td>sociologist</td><td>58.7% (0.9%)</td></tr><tr><td>prof</td><td>57.6% (2.6%)</td></tr><tr><td>adjunct</td><td>57.0% (-0.9%)</td></tr><tr><td>assistant</td><td>57.0% (-5.5%)</td></tr><tr><td>psychology</td><td>56.9% (2.9%)</td></tr><tr><td>university</td><td>56.6% (-4.3%)</td></tr></table></td><td><table><tr><td>professor</td><td>94.4% (0.7%)</td></tr><tr><td>lecturer</td><td>75.5% (7.6%)</td></tr><tr><td>emeritus</td><td>66.8% (-2.5%)</td></tr><tr><td>dean</td><td>66.6% (2.7%)</td></tr><tr><td>researcher</td><td>66.1% (2.4%)</td></tr><tr><td>graduate</td><td>65.8% (4.5%)</td></tr><tr><td>associate</td><td>64.6% (0.7%)</td></tr><tr><td>scholar</td><td>64.5% (2.1%)</td></tr><tr><td>professors</td><td>63.8% (1.1%)</td></tr><tr><td>scientist</td><td>63.8% (2.5%)</td></tr><tr><td>faculty</td><td>63.4% (4.4%)</td></tr><tr><td>doctoral</td><td>62.6% (2.5%)</td></tr><tr><td>assistant</td><td>62.5% (5.5%)</td></tr><tr><td>sociology</td><td>61.8% (-0.2%)</td></tr><tr><td>university</td><td>61.0% (4.3%)</td></tr><tr><td>doctorate</td><td>60.8% (0.0%)</td></tr><tr><td>student</td><td>58.8% (5.5%)</td></tr><tr><td>undergraduate</td><td>58.2% (2.0%)</td></tr><tr><td>adjunct</td><td>57.9% (0.9%)</td></tr><tr><td>sociologist</td><td>57.8% (-0.9%)</td></tr></table></td></tr></table>

Looking for gender bias in 'professor' for 'man' and 'woman': 100%|█████████▉| 1343765/1344233 [00:24<00:00, 55010.67 words/s]
Getting similarity difference between roles: 100%|██████████| 247512/247512 [00:00<00:00, 394307.01roles/s]


<table><tr><th>Role</th><th>man</th><th>woman</th></tr><tr><td>professor</td><td><table><tr><td>professor</td><td>83.3% (-1.0%)</td></tr><tr><td>emeritus</td><td>65.6% (10.0%)</td></tr><tr><td>lecturer</td><td>61.5% (-4.7%)</td></tr><tr><td>scholar</td><td>58.9% (4.8%)</td></tr><tr><td>dean</td><td>58.4% (0.6%)</td></tr><tr><td>scientist</td><td>58.0% (4.5%)</td></tr><tr><td>associate</td><td>56.0% (-2.3%)</td></tr><tr><td>professors</td><td>55.7% (-1.2%)</td></tr><tr><td>alumnus</td><td>54.6% (9.2%)</td></tr><tr><td>researcher</td><td>53.6% (-8.2%)</td></tr><tr><td>prof</td><td>52.2% (4.3%)</td></tr><tr><td>graduate</td><td>52.1% (-8.9%)</td></tr><tr><td>economics</td><td>52.0% (3.6%)</td></tr><tr><td>sociology</td><td>51.7% (-6.6%)</td></tr><tr><td>faculty</td><td>51.7% (-5.6%)</td></tr><tr><td>historian</td><td>51.6% (7.4%)</td></tr><tr><td>physicist</td><td>50.6% (5.9%)</td></tr><tr><td>doctorate</td><td>50.6% (-7.0%)</td></tr><tr><td>biologist</td><td>50.5% (2.0%)</td></tr><tr><td>economist</td><td>50.4% (6.4%)</td></tr></table></td><td><table><tr><td>professor</td><td>84.2% (1.0%)</td></tr><tr><td>lecturer</td><td>66.3% (4.7%)</td></tr><tr><td>researcher</td><td>61.9% (8.2%)</td></tr><tr><td>graduate</td><td>61.0% (8.9%)</td></tr><tr><td>doctoral</td><td>58.8% (8.3%)</td></tr><tr><td>sociology</td><td>58.3% (6.6%)</td></tr><tr><td>associate</td><td>58.3% (2.3%)</td></tr><tr><td>dean</td><td>57.9% (-0.6%)</td></tr><tr><td>doctorate</td><td>57.6% (7.0%)</td></tr><tr><td>faculty</td><td>57.3% (5.6%)</td></tr><tr><td>university</td><td>56.9% (9.2%)</td></tr><tr><td>professors</td><td>56.9% (1.2%)</td></tr><tr><td>psychologist</td><td>56.6% (13.5%)</td></tr><tr><td>assistant</td><td>56.5% (6.7%)</td></tr><tr><td>student</td><td>56.4% (13.1%)</td></tr><tr><td>alumna</td><td>56.3% (30.1%)</td></tr><tr><td>emeritus</td><td>55.6% (-10.0%)</td></tr><tr><td>librarian</td><td>54.9% (22.1%)</td></tr><tr><td>educator</td><td>54.7% (16.0%)</td></tr><tr><td>sociologist</td><td>54.6% (5.7%)</td></tr></table></td></tr></table>

Notable points:
- For Doctors, Nurses and midwives appear to be more female weighted and urologists appear to be more male weighted, despite male and female versions having the same title.
- For dancers, female roles feature ballet, burlesque and gymnast, whereas these are less male weighted
- Some fields in general seem to be male-dominated, with overall slightly higher percentages across the board

More analysis required to do direct comparisons rather than just a quick "head" of the data. Should also evaluate the other SpaCy models.